To use this retriever, we first need to write the Cypher query to specify exactly what 
additional information to fetch along with the node retrieved through vector search. 
For example, to retrieve actor information along with the Movie nodes, we can use the following query:

In [ ]:
retrieval_query = """
MATCH
(actor:Actor)-[:ACTED_IN]->(node)
RETURN
node.title AS movie_title,
node.plot AS movie_plot, 
collect(actor.name) AS actors;
"""


The node variable in this query is a reference to the nodes retrieved through 
the initial vector search step, which here are Movie nodes. 
This query finds all of the actors who have acted in each of the movies and returns their names 
along with the title and plot of the movie.

In [ ]:
from neo4j import GraphDatabase
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
#from neo4j_graphrag.embeddings.ollama import OllamaEmbeddings
from neo4j_graphrag.retrievers import VectorCypherRetriever

### DO THIS IN UTILS TO DEFINE THE RETRIEVER ###
driver = GraphDatabase.driver(URI, auth=AUTH) #define your specific driver
embedder = OpenAIEmbeddings(model="text-embedding-ada-002")
vc_retriever = VectorCypherRetriever(
    driver,
    index_name="moviePlotsEmbedding",
    embedder=embedder,
    retrieval_query=retrieval_query,
)


Then, when passing a query text, the output should be like this:

In [ ]:
query_text = "Who were the actors in the movie about the magic jungle board game?"
retriever_result = vc_retriever.search(query_text=query_text, top_k=3)

# items=[
#   RetrieverResultItem(content="<Record
#     movie_title='Jumanji'
#     movie_plot='When two kids find and play a magical board game, they release a man trapped for decades in it and a host of dangers that can only be stopped by finishing the game.'
#     actors=['Robin Williams', 'Bradley Pierce', 'Kirsten Dunst', 'Jonathan Hyde']>",
#     metadata=None),
#   RetrieverResultItem(content="<Record
#     movie_title='Welcome to the Jungle'
#     movie_plot='A company retreat on a tropical island goes terribly awry.'
#     actors=['Jean-Claude Van Damme', 'Adam Brody', 'Rob Huebel', 'Kristen Schaal']>",
#     metadata=None),
#   RetrieverResultItem(content='<Record
#     movie_title='Last Mimzy, The'
#     movie_plot='Two siblings begin to develop special talents after they find a mysterious box of toys. Soon the kids, their parents, and even their teacher are drawn into a strange new world and find a task ahead of them that is far more important than any of them could imagine!'
#     actors=['Joely Richardson', 'Rhiannon Leigh Wryn', 'Timothy Hutton', "Chris O'Neil"]>',
#     metadata=None)
# ]
# metadata={'__retriever': 'VectorCypherRetriever'}


And this is the pipeline, easy:

In [ ]:
from neo4j_graphrag.llm import OpenAILLM
#from neo4j_graphrag.llm import OllamaLLM
from neo4j_graphrag.generation import GraphRAG

# LLM
# Note: the OPENAI_API_KEY must be in the env vars
llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})
#llm = OpenAILLM(model_name="llama3.1:8b", model_params={"temperature": 0})

# Initialize the RAG pipeline
rag = GraphRAG(retriever=vc_retriever, llm=llm)

# Query the graph
query_text = "Who were the actors in the movie about the magic jungle board game?"
response = rag.search(query=query_text, retriever_config={"top_k": 3})
print(response.answer)

Now an example to run graphrag with pinecone (generated by gpt, check for correctness)

In [ ]:
from neo4j import GraphDatabase
from langchain.schema import Document
from langchain_ollama import OllamaEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from neo4j_graphrag.retrievers import VectorCypherRetriever

# Secrets & config
NEO4J_URI = "neo4j+s://<YOUR_AURA_URI>"
NEO4J_AUTH = ("<USER>", "<PASSWORD>")
NEO4J_DB = "<DATABASE>"
PINECONE_API_KEY = "<PINECONE_KEY>"
INDEX_NAME = "<PINECONE_INDEX>"
EMBEDDER_MODEL = "<OLLAMA_MODEL>"

# Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

# Pinecone setup
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)
embed_model = OllamaEmbeddings(model=EMBEDDER_MODEL, base_url="http://127.0.0.1:11435")
vectorstore = PineconeVectorStore(index=index, embedding=embed_model)


In [ ]:
# Define the retriever
retriever = VectorCypherRetriever(
    vectorstore=vectorstore,     # Pinecone vector store
    driver=driver,               # Neo4j driver
    index_name="moviePlotsEmbedding",
    embedder=embedder,
    retrieval_query=retrieval_query
)

In [ ]:
query = "Which Fire-type Pokémon evolve from Charmander?"


# for doc in results:
#     uid = doc.metadata["pokemon_uid"]
#     name = doc.metadata["pokemon_name"]
#     print(f"Chunk from Pokémon UID={uid}, name={name}")
#     print(doc.page_content[:200], "...")  # first 200 chars
#     print("-"*50)

retriever_result = vc_retriever.search(query_text=query, top_k=5)